In [1]:
import os
import json
import numpy as np
from collections import defaultdict
from tqdm import tqdm

# ===== 경로 설정 =====
BBOX_JSON_ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/02.라벨링데이터/TL/이미지"
OUT_ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/02.라벨링데이터/TL/이미지/bbox_motion"
os.makedirs(OUT_ROOT, exist_ok=True)


# ===== bbox 문자열 파싱 =====
def parse_bbox(s):
    return list(map(float, s.split(",")))


# ===== bbox sequence → motion feature =====
def bbox_to_motion(bboxes):
    feats = []
    prev_cx, prev_cy = None, None

    for x1, y1, x2, y2 in bboxes:
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        w = x2 - x1
        h = y2 - y1

        if prev_cx is None:
            dx, dy = 0.0, 0.0
        else:
            dx = cx - prev_cx
            dy = cy - prev_cy

        feats.append([cx, cy, w, h, dx, dy])
        prev_cx, prev_cy = cx, cy

    return feats  # (T, 6)


# ===== 모든 bbox JSON 수집 =====
scene_frames = defaultdict(list)

json_files = []
for root, _, files in os.walk(BBOX_JSON_ROOT):
    for f in files:
        if f.lower().endswith(".json"):
            json_files.append(os.path.join(root, f))

print("총 bbox JSON 수:", len(json_files))


# ===== scene 단위로 묶기 =====
for jf in tqdm(json_files, desc="Collect bbox frames"):
    with open(jf) as f:
        data = json.load(f)

    scene = data["metadata"]["scene_id"]
    fname = data["metadata"]["file_name"]
    bbox = parse_bbox(data["bboxdata"]["bbox_location"])

    scene_frames[scene].append((fname, bbox))


# ===== scene별 bbox motion 생성 =====
for scene, frames in tqdm(scene_frames.items(), desc="Build bbox motion"):
    # 프레임 순서 정렬 (I001 ~ I010)
    frames.sort(key=lambda x: x[0])

    bboxes = [b for _, b in frames]
    motion = bbox_to_motion(bboxes)

    out_path = os.path.join(OUT_ROOT, f"{scene}.json")
    with open(out_path, "w") as f:
        json.dump(
            {
                "scene_id": scene,
                "bbox_motion": motion
            },
            f,
            indent=2
        )

print("🔥 bbox motion scene JSON 생성 완료")

총 bbox JSON 수: 41280


Build bbox motion: 100%|██████████| 4128/4128 [00:54<00:00, 75.34it/s]

🔥 bbox motion scene JSON 생성 완료


In [2]:
import os
import json
import numpy as np
from collections import defaultdict
from tqdm import tqdm

# ===== 경로 설정 =====
BBOX_JSON_ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Validation/02.라벨링데이터/VL/이미지"
OUT_ROOT = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Validation/02.라벨링데이터/VL/이미지/bbox_motion"
os.makedirs(OUT_ROOT, exist_ok=True)


# ===== bbox 문자열 파싱 =====
def parse_bbox(s):
    return list(map(float, s.split(",")))


# ===== bbox sequence → motion feature =====
def bbox_to_motion(bboxes):
    feats = []
    prev_cx, prev_cy = None, None

    for x1, y1, x2, y2 in bboxes:
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        w = x2 - x1
        h = y2 - y1

        if prev_cx is None:
            dx, dy = 0.0, 0.0
        else:
            dx = cx - prev_cx
            dy = cy - prev_cy

        feats.append([cx, cy, w, h, dx, dy])
        prev_cx, prev_cy = cx, cy

    return feats  # (T, 6)


# ===== 모든 bbox JSON 수집 =====
scene_frames = defaultdict(list)

json_files = []
for root, _, files in os.walk(BBOX_JSON_ROOT):
    for f in files:
        if f.lower().endswith(".json"):
            json_files.append(os.path.join(root, f))

print("총 bbox JSON 수:", len(json_files))


# ===== scene 단위로 묶기 =====
for jf in tqdm(json_files, desc="Collect bbox frames"):
    with open(jf) as f:
        data = json.load(f)

    scene = data["metadata"]["scene_id"]
    fname = data["metadata"]["file_name"]
    bbox = parse_bbox(data["bboxdata"]["bbox_location"])

    scene_frames[scene].append((fname, bbox))


# ===== scene별 bbox motion 생성 =====
for scene, frames in tqdm(scene_frames.items(), desc="Build bbox motion"):
    # 프레임 순서 정렬 (I001 ~ I010)
    frames.sort(key=lambda x: x[0])

    bboxes = [b for _, b in frames]
    motion = bbox_to_motion(bboxes)

    out_path = os.path.join(OUT_ROOT, f"{scene}.json")
    with open(out_path, "w") as f:
        json.dump(
            {
                "scene_id": scene,
                "bbox_motion": motion
            },
            f,
            indent=2
        )

print("🔥 bbox motion scene JSON 생성 완료")

총 bbox JSON 수: 5920


Build bbox motion: 100%|██████████| 592/592 [00:07<00:00, 82.10it/s]

🔥 bbox motion scene JSON 생성 완료
